In [0]:
from __future__ import division
import os
from collections import OrderedDict,Counter
import pandas as pd
import numpy as np
import vcf
from operator import itemgetter
import random
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.ticker as mtick
%matplotlib inline
import math
from scipy.stats import ks_2samp
from scipy.stats import anderson_ksamp
from os import listdir as ls
from os import path as op
from scipy.stats import spearmanr
import skbio
from scipy.stats import pearsonr
import shutil
import pickle

In [0]:
#get the hierftrans for all SNPS
#z12 file created in 06_pca.ipyn
filE = '/home/lindb/wbp/OutFLANK/imputed_z12_maf_swp_trans_z12.txt'
imp012 = pd.read_csv(filE,header=0,index_col=0,sep="\t")
imp012.head()

In [0]:
#get pop assignment for each samp
filE = '/home/lindb/wbp/sampsTOpop.txt'
stp = pd.read_csv(filE,header=0,index_col='sampID',sep="\t")
stp.sort_index(inplace=True)
stp.head()

In [0]:
#list of samps per pop
stpdict= OrderedDict()
for row in stp.index:
    pop = stp.loc[row,'pop']
    if not pop in stpdict.keys():
        stpdict[pop] = []
    stpdict[pop].append(row)
stpdict

In [0]:
#counts per pop
popdict = Counter()
for row in stp.index:
    pop = stp.loc[row,'pop']
    if not pop in popdict.keys():
        popdict[pop] = 0
    popdict[pop] += 1
popdict

In [0]:
samps = stp.index.tolist()

In [0]:
#reassign individuals to a random population
randpopdict = OrderedDict()
for pop in popdict:
    randpopdict[pop] = []
    randpopdict[pop] = random.sample(samps,popdict[pop])
    samps = set(samps) - set(randpopdict[pop])

In [0]:
#make sure all samples were assigned to pops
sums = 0
for pop in randpopdict:
    sums = sums + len(randpopdict[pop])
sums == sum(popdict.values())

In [0]:
#make sure pop size is the same
for pop in randpopdict:
    print pop,len(randpopdict[pop]),popdict[pop]

In [0]:
#see how many of the individuals stayed in the same pop
for pop in popdict:
    print pop, len(set(randpopdict[pop]).intersection(set(stpdict[pop])))

In [0]:
#make new stp dataframe
stpnew = pd.DataFrame(stp)
for samp in stpnew.index:
    stpnew.loc[samp,'pop'] = [pop for pop,samplist in randpopdict.items() if samp in samplist][0]
stpnew.sort_index(inplace=True)
stpnew.head()

In [0]:
merged = pd.merge(imp012,stpnew,left_index=True,right_index=True)
cols = ['pop'] + [col for col in merged.columns if 'NODE' in col]
merged = merged[cols]
merged.sort_index(inplace=True)
merged.head()

In [0]:
merged.shape

In [0]:
cols = [col for col in merged.columns if 'NODE' in col]
snpmat = merged[cols]
snpmat.head()

In [0]:
pops = pd.DataFrame(merged['pop'].tolist())
pops.head()

In [0]:
DIR = "/home/lindb/wbp/outflank_null"
if not op.exists(DIR):
    os.makedirs(DIR)
    print "True"

In [0]:
filE = '/home/lindb/wbp/outflank_null/SNPmat_HEADERIDX.txt'
filE2 = '/home/lindb/wbp/outflank_null/SNPmat_noHEADERIDX.txt'
print 'making 1st snpmat' #so I don't have to watch ls -lt
snpmat.to_csv(filE,header=True,index=True,sep="\t")
print 'making 2nd snpmat'
snpmat.to_csv(filE2,header=None,index=False,sep="\t")

popfile = '/home/lindb/wbp/outflank_null/SNPmat_popNames.txt'
print 'making popfile'
pops.to_csv(popfile,header=False,index=False,sep="\t")

print 'making locfile'
locfile = '/home/lindb/wbp/outflank_null/SNPmat_locusNames.txt'
cols = pd.DataFrame(snpmat.columns)
cols.to_csv(locfile,header=False,index=False,sep="\t")

In [0]:
text = '''
library(OutFLANK)
library(data.table)

SNPmat = data.frame(fread('/home/lindb/wbp/outflank_null/SNPmat_noHEADERIDX.txt',header=F,sep="\\t"))

locusNames = read.csv('/home/lindb/wbp/outflank_null/SNPmat_locusNames.txt',header=F,sep="\\t")

popNames = read.csv('/home/lindb/wbp/outflank_null/SNPmat_popNames.txt',header=F,sep="\\t")

FstDataFrame = MakeDiploidFSTMat(SNPmat,locusNames,popNames)

out = OutFLANK(FstDataFrame = FstDataFrame,NumberOfSamples = 8)

df = out$results

outliers = df[which(df$OutlierFlag == 'TRUE'),]

loci = outliers$LocusName

write.table(df,'/home/lindb/wbp/outflank_null/OutFLANK_results.txt',row.names=F,col.names=T,sep='\\t')

write.table(loci,'/home/lindb/wbp/outflank_null/OutFLANK_snps.txt',row.names=F,sep='\\t')

print("DONE!")
'''
filE = "/home/lindb/wbp/outflank_null/do_the_outflank.R"
with open(filE,'w') as o:
    o.write("%s" % text)

In [0]:
#tmux running on godel35
#sourced the R script, waiting to finish

In [0]:
loc = pd.read_csv("/home/lindb/wbp/outflank_null/OutFLANK_snps.txt",sep="\t")
loc.head()

In [0]:
len(loc.index)

In [0]:
locold = pd.read_csv("/home/lindb/wbp/OutFLANK/OutFLANK_snps.txt",sep="\t")

In [0]:
len(set(loc['x']).intersection(set(locold['x'])))

# Make other 99 runs of null outflank

In [0]:
for i in range(99):
    DIR = "/home/lindb/wbp/outflank_null/null_%s" % (str(i+2)).zfill(3)
    if not op.exists(DIR):
        os.makedirs(DIR)

In [0]:
%time z12 = pd.read_csv('/home/lindb/wbp/OutFLANK/imputed_z12_maf_swp_trans_z12.txt',header=0,index_col=0,sep="\t")
filE = '/home/lindb/wbp/OutFLANK/imputed_z12_maf_swp_trans_z12.pkl'
with open(filE,'wb') as o:
    pickle.dump(z12,o,pickle.HIGHEST_PROTOCOL)

In [0]:
for i in range(99):
    text = '''
from __future__ import division
import os
from collections import OrderedDict,Counter
import pandas as pd
import numpy as np
import vcf
from operator import itemgetter
import random
import math
from os import listdir as ls
from os import path as op
import pickle

DIR = "/home/lindb/wbp/outflank_null/null_%%s" %% (str(%s)).zfill(3)
print DIR


#get the hierftrans for all SNPS
#z12 file created in 06_pca.ipyn
filE = '/home/lindb/wbp/OutFLANK/imputed_z12_maf_swp_trans_z12.pkl'
imp012 = pickle.load(open(filE,'rb'))
print "loaded imp"

#get pop assignment for each samp
filE = '/home/lindb/wbp/sampsTOpop.txt'
stp = pd.read_csv(filE,header=0,index_col='sampID',sep="\\t")
stp.sort_index(inplace=True)
print "made stp df"

#list of samps per pop
stpdict= OrderedDict()
for row in stp.index:
    pop = stp.loc[row,'pop']
    if not pop in stpdict.keys():
        stpdict[pop] = []
    stpdict[pop].append(row)
print "made stpdict"

#counts per pop
popdict = Counter()
for row in stp.index:
    pop = stp.loc[row,'pop']
    if not pop in popdict.keys():
        popdict[pop] = 0
    popdict[pop] += 1
print "made popdict"

samps = stp.index.tolist()

#reassign individuals to a random population
randpopdict = OrderedDict()
for pop in popdict:
    randpopdict[pop] = []
    randpopdict[pop] = random.sample(samps,popdict[pop])
    samps = set(samps) - set(randpopdict[pop])
print "reassigned individuals to random pop"

#make sure all samples were assigned to pops
sums = 0
for pop in randpopdict:
    sums = sums + len(randpopdict[pop])
assert sums == sum(popdict.values())

#make sure pop size is the same
for pop in randpopdict:
    assert len(randpopdict[pop]) == popdict[pop]

#make new stp dataframe
stpnew = pd.DataFrame(stp)
for samp in stpnew.index:
    stpnew.loc[samp,'pop'] = [pop for pop,samplist in randpopdict.items() if samp in samplist][0]
stpnew.sort_index(inplace=True)
print "made stpnew"

#merge the dataframes
merged = pd.merge(imp012,stpnew,left_index=True,right_index=True)
cols = ['pop'] + [col for col in merged.columns if 'NODE' in col]
merged = merged[cols]
merged.sort_index(inplace=True)
print "merged"

#make snpmat
cols = [col for col in merged.columns if 'NODE' in col]
snpmat = merged[cols]

pops = pd.DataFrame(merged['pop'].tolist())

filE = op.join(DIR,'SNPmat_HEADERIDX_%s.txt')
filE2 = op.join(DIR,'SNPmat_noHEADERIDX_%s.txt')

snpmat.to_csv(filE,header=True,index=True,sep="\\t")
print "made snpmat"

snpmat.to_csv(filE2,header=None,index=False,sep="\\t")
print "made second snpmat"

popfile = op.join(DIR,'SNPmat_popNames_%s.txt')

pops.to_csv(popfile,header=False,index=False,sep="\\t")
print "made popfile"

locfile = op.join(DIR,'SNPmat_locusNames_%s.txt')
cols = pd.DataFrame(snpmat.columns)
cols.to_csv(locfile,header=False,index=False,sep="\\t")
print "done"
''' % ((i+2),str(i+2).zfill(3),str(i+2).zfill(3),str(i+2).zfill(3),str(i+2).zfill(3))
    DIR = "/home/lindb/wbp/outflank_null/make_infiles"
    if not op.exists(DIR):
        os.makedirs(DIR)
    filE = op.join(DIR,"make_infiles_%s.py") % (str(i+2).zfill(3))
    with open(filE,'w') as o:
        o.write("%s" % text)

In [0]:
DIR = "/home/lindb/wbp/outflank_null/make_infiles"
files = [op.join(DIR,f) for f in ls(DIR) if '.py' in f]
len(files)

In [0]:
from os import listdir as ls
from os import path as op

In [0]:
DIR = '/home/lindb/wbp/outflank_null/'
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d]
rDIR = op.join("/home/lindb/wbp/outflank_null",'r_files')
for d in dirs:
    num = op.basename(d).split("_")[1]
    if not num == '001':
        text = '''
library(OutFLANK)
library(data.table)

SNPmat = data.frame(fread('%s/SNPmat_noHEADERIDX_%s.txt',header=F,sep="\\t"))

locusNames = read.csv('%s/SNPmat_locusNames_%s.txt',header=F,sep="\\t")

popNames = read.csv('%s/SNPmat_popNames_%s.txt',header=F,sep="\\t")

FstDataFrame = MakeDiploidFSTMat(SNPmat,locusNames,popNames)

out = OutFLANK(FstDataFrame = FstDataFrame,NumberOfSamples = 8)

df = out$results

outliers = df[which(df$OutlierFlag == 'TRUE'),]

loci = outliers$LocusName

write.table(df,'%s/OutFLANK_results_%s.txt',row.names=F,col.names=T,sep='\\t')

write.table(loci,'%s/OutFLANK_snps_%s.txt',row.names=F,sep='\\t')

print("DONE!")
''' % (d,num,
       d,num,
       d,num,
       d,num,
       d,num)
        filE = op.join(rDIR,"null_outflank_%s.R") % num
        if not op.exists(rDIR):
            os.makedirs(rDIR)
        with open(filE,'w') as o:
            o.write("%s" % text)

In [0]:
DIR = "/home/lindb/wbp/outflank_null/make_infiles"
files = [op.join(DIR,f) for f in ls(DIR) if '.py' in f]
len(files)

In [0]:
DIR = "/home/lindb/wbp/outflank_null/make_infiles"
files = [op.join(DIR,f) for f in ls(DIR) if '.py' in f]
count = 0
scount = 0
for f in files:
    num = op.basename(f).split("_")[2].split(".")[0]
    if count == 0:
        text = '''#!/usr/bin/bash
#$ -N inf%s
#$ -S /bin/bash
#$ -V
#$ -j y

cd /home/lindb/wbp/outflank_null/make_infiles/
python < make_infiles_%s.py
R --no-save < /home/lindb/wbp/outflank_null/r_files/null_outflank_%s.R
''' % (scount,num,num)
    else:
        newtext = '''
cd /home/lindb/wbp/outflank_null/make_infiles/
python < make_infiles_%s.py
R --no-save < /home/lindb/wbp/outflank_null/r_files/null_outflank_%s.R
''' % (num,num)
        text = text + newtext
    count += 1
    if count == 10 or (count ==9 and scount == 9):
        count = 0
        
        filE = op.join(DIR,"run_them_all_%s.sh" % scount)
        with open(filE,'w') as o:
            o.write("%s" % text)
        scount += 1

In [0]:
#using tmux on godel06 and godel97 to run the stupid null files

In [0]:
DIR = '/home/lindb/wbp/outflank_null/'
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d]
for d in dirs:
    files = [op.join(d,f) for f in ls(d)]
    if len(files) > 0:
        if len(files) == 6:
            print op.basename(d),"done"
        else:
            print op.basename(d),len(files)

# retrieve significant SNPs

In [0]:
#see how many sig snps I'm getting
DIR = "/home/lindb/wbp/outflank_null/"
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d]
sigs = []
for d in dirs:
    files = [op.join(d,f) for f in ls(d) if 'snps' in f.split("_")[:-1]]
    if len(files) >0:
        df = pd.read_csv(files[0])
        sigs.append(len(df.index))
    else:
        print op.basename(d)
plt.hist(sigs)

In [0]:
np.mean(sigs), np.max(sigs), np.median(sigs),len(sigs)

# make pickle files for easier upload

In [0]:
DIR = '/home/lindb/wbp/outflank_null/make_pickles'
if not op.exists(DIR):
    os.makedirs(DIR)

In [0]:
DIR = '/home/lindb/wbp/outflank_null/'
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d]
for d in dirs:
    num = op.basename(d).split("_")[1]
    filE = op.join(d,'SNPmat_HEADERIDX_%s.txt' % num)
    if op.exists(filE):
        filE2 = op.join(d,'SNPmat_HEADERIDX_%s.pkl' % num)
        text = '''
from __future__ import division
import os
from collections import OrderedDict,Counter
import pandas as pd
import numpy as np
import vcf
from operator import itemgetter
import random
import math
from os import listdir as ls
from os import path as op
import pickle

df = pd.read_csv('%s',header=0,index_col=0,sep="\\t")       ###########
with open('%s','wb') as o:                                  ###########
    pickle.dump(df,o,pickle.HIGHEST_PROTOCOL)
''' % (filE,
       filE2
      )
        filE3 = op.join(DIR,'make_pickles/%s.py' % num)
        with open(filE3,'w') as o:
            o.write("%s" % text)

In [0]:
DIR = '/home/lindb/wbp/outflank_null/make_pickles/'
files = [op.join(DIR,f) for f in ls(DIR) if 'py' in f]
#len(files)
for f in files:
    num = op.basename(f).split(".")[0]
    text = '''#!/usr/bin/bash
#$ -N inf%s
#$ -S /bin/bash
#$ -V
#$ -j y

cd /home/lindb/wbp/outflank_null/make_pickles/
python %s
''' % (num,f)
    filE = op.join(DIR,"run_%s.sh" % num)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [0]:
DIR = '/home/lindb/wbp/outflank_null/make_pickles/'
files = [op.join(DIR,f) for f in ls(DIR) if 'sh' in f]
for f in files:
#    !qsub $f
# i executed this in terminal

In [0]:
#see how many pickle files
DIR = '/home/lindb/wbp/outflank_null/'
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d]
files = []
for d in dirs:
    num = op.basename(d).split("_")[1]
    filE = op.join(d,'SNPmat_HEADERIDX_%s.pkl' % num) 
    if op.exists(filE):
        files.append(filE)
len(files)

# covariances

In [0]:
#make glob dictionary a pickle for quicker loading
filE  = '/home/lindb/wbp/OutFLANK/global_mafs.txt'
df = pd.read_csv(filE,header=0,index_col=0,sep="\t")
filE2 = '/home/lindb/wbp/OutFLANK/global_mafs.pkl'
#with open(filE2,'wb') as o:
    pickle.dump(df,o,pickle.HIGHEST_PROTOCOL)

### get pop-level maf, calc focal dij, create 1000 sets of null loci

In [0]:
DIR = "/home/lindb/wbp/outflank_null/"
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d]
for d in dirs:
    num = op.basename(d).split("_")[1]
    text = '''
from __future__ import division
import os
from collections import OrderedDict,Counter
import pandas as pd
import numpy as np
import vcf
from operator import itemgetter
import random
import math
from os import listdir as ls
from os import path as op
import skbio
import shutil
import pickle


#get global minor allele frequencies
print "getting global maf"
glob = pickle.load(open("/home/lindb/wbp/OutFLANK/global_mafs.pkl","rb"))

############################### Get pop-level MAFs #################################################
#get 012 data with population assignments
print "uploading 012"
filE = "%s"                                                          ###############################
data = pickle.load(open(filE,"rb"))

print "merging data frames"
m = pd.read_csv("%s", header=None,index_col=None,sep="\\t")          ###############################
data["pop"] = m[0].tolist()
cols = ["pop"] + [col for col in data.columns if "NODE" in col]
data = data[cols]

#get a list of samps for each population
print "getting samp lists per pop - these samps have been randomly assigned"
pops = np.unique(data["pop"]).tolist()
popDict = OrderedDict()
for samp in data.index:
    pop = data.loc[samp,"pop"]
    if not pop in popDict.keys():
        popDict[pop] = []
    popDict[pop].append(samp)

#get dataframes for each pop based on samples assigned to that pop
print "getting pop-level dfs"
popDictImp = OrderedDict()
for pop in sorted(pops):
    popDictImp[pop]  = data[data.index.isin(popDict[pop])]

#get major and minor allele counts per pop
print "determining mjr mnr counts"
filE = "%s"                                                          ###############################
if not op.exists(filE):
    locCount = 0
    Dict = OrderedDict()
    for locus in data.columns[data.columns != 'pop']:
        Dict[locus] = OrderedDict()
        popCount = 0
        for pop in sorted(pops):

            zero = popDictImp[pop][locus].tolist().count(0) #count the first homozygotes
            one = popDictImp[pop][locus].tolist().count(1) #count the heterozygotes
            two = popDictImp[pop][locus].tolist().count(2) #count the second homozygotes        

            #012 counts global minor allele
            A1 = 2*zero + one
            A2 = 2*two + one

            #no need to distinguish A1 > A2, since MAF within pop refers to global minor allele

            if len(Dict[locus].keys()) == 0:
                Dict[locus]["A1"] = OrderedDict()
                Dict[locus]["A2"] = OrderedDict()
            Dict[locus]["A1"][pop] = A1
            Dict[locus]["A2"][pop] = A2
            #break
        locCount += 1
        if locCount %% 1000 == 0:
            print locCount

    print "writing mjr mnr count file"
    with open(filE, "w") as o:
        line = "\\t".join([str(pop) for pop in Dict[Dict.keys()[0]][Dict[Dict.keys()[0]].keys()[0]].keys()]) + str("\\n")
        o.write("%%s" %% line)
        for locus in Dict.keys():
            for allele in Dict[locus].keys():
                line = str(locus) + "\\t" + "\\t".join([str(x) for x in Dict[locus][allele].values()]) + str("\\n")
                #print locus,allele,line
                o.write("%%s" %% line)

else:
    print "mjr mnr count file already exists, skipping calculation"

#get allele counts by pop - first locus = counts of 0 allele, second = counts of 2 allele
    #012 counts global minor allele
counts = pd.read_csv(filE,header=0,index_col=0,sep="\\t")

loci = np.unique(counts.index).tolist()
print "getting pop-level maf"
filE = "%s"                                                           ###############################
if not op.exists(filE):
    loccount = 0
    mafDict = OrderedDict()
    for locus in loci:
        mafDict[locus] = OrderedDict()
        DATA = pd.DataFrame(counts.loc[locus,:])
        DATA.index = ["major","minor"]
        for pop in DATA.columns:
            MAF = DATA.loc["minor",pop]/sum(DATA[pop]) #get allele freq corresponding to global minor allele
            mafDict[locus][pop] = MAF
        loccount += 1
        if loccount %% 1000 == 0:
            print loccount

    print "writing pop-level maf"
    with open(filE,"w") as o:
        text = "\\t".join([x for x in mafDict[mafDict.keys()[0]].keys()]) + "\\n"
        o.write("%%s" %% text)
        print text
        count = 0
        for locus in mafDict.keys():
            text = locus + "\\t" + "\\t".join([str(x) for x in mafDict[locus].values()]) + "\\n"
            o.write("%%s" %% text)
            count += 1
else:
    print "pop-level maf already created, skipping calculation"

print "reading pop-level maf"
impMAF = pd.read_csv(filE,header=0,index_col=0,sep="\\t")
##############################################################################################

#get outlier snps
print "getting outlier snps"
df = pd.read_csv("%s",header=0,sep="\\t")                             ###############################
outliersnps = df["x"].tolist()

#do pairwise to get Dij for outliers
print "getting pop counts"
popcounts = OrderedDict()
for pop in pops:
    popcounts[pop] = len(popDict[pop])

print "getting dij for outliers"
dijDict = OrderedDict() 
icount = 0
for i,locusi in enumerate(outliersnps):
    dijDict[locusi] = OrderedDict()
    qi = glob[locusi] #global maf
    
    for j,locusj in enumerate(outliersnps):
        if i > j: #i=row, j=col : lower triangle 
            qj = glob[locusj] #global maf
            
            sums = 0
            for pop in impMAF.columns:
                qik = impMAF.loc[locusi,pop] #get pop maf
                qjk = impMAF.loc[locusj,pop] #get pop maf
                nk = popcounts[pop]
                
                sums += (nk/244)*((qik*qjk)-(qi*qj))

            dijDict[locusi][locusj] = sums
        else:
            dijDict[locusi][locusj] = np.nan
    icount += 1
    if icount %% 10 == 0:
        print icount
#write out the file
print "writing dij for outliers"
rowcount = 0
filE = "%s"                                                          ###############################
with open(filE,"w") as o:
    key0 = dijDict.keys()[0]
    line = "\\t".join(dijDict[key0].keys()) + str("\\n")
    o.write("%%s" %% line)
    for locusi in dijDict.keys():
        line = str(locusi)+"\\t"+"\\t".join([str(x) for x in dijDict[locusi].values()]) + str("\\n")
        o.write("%%s" %% line)
dvals = pd.read_csv(filE,header=0,index_col=0,sep="\\t")

#get global expected heterozygosity and bins
print "getting global het bins"
filE = "/home/lindb/wbp/OutFLANK/Hexp_by_snp_withbins.txt"
H = pd.read_csv(filE, header=0,index_col=0,sep="\\t")
H.index = [snp for snp in H["locus"].tolist()]

#get a dataframe with the outlier loci and their bins
print "getting bins for outlier data"
outlierdata = pd.DataFrame(H[H["locus"].isin(outliersnps)])
outlierdata.index = [snp for snp in outlierdata["locus"].tolist()]

print "determining non-outliers for null draws"
nonsigs = set(H.index.tolist()) - set(outlierdata.index.tolist())
nonsigs = [x for x in nonsigs]
nonsigdata = pd.DataFrame(H[H["locus"].isin(nonsigs)])

#how many random snps from each bin?
print "creating binCounter for outlier loci"
binCounter = Counter()
for row in outlierdata.index:
    binCounter[outlierdata.loc[row,"bin"]] += 1

folder = "%s"                                                       ###############################
if not op.exists(folder):
    os.makedirs(folder)

#make 1000 dataframes of random snps of equal size
print "making 1000 dfs of random snps"
for i in range(1000):
    snps = []        
    for binn in binCounter.keys():
        data = nonsigdata[nonsigdata["bin"] == binn]

        [snps.append(snp) for snp in random.sample(data.index,binCounter[binn])]

    filE = op.join(folder,"outflank_null_%%s_randsnps.txt" %% (str(i).zfill(4)))
    df = pd.DataFrame(snps)
    df.to_csv(filE,header=False,index=False,sep="\\t")
''' % (op.join(d,'SNPmat_HEADERIDX_%s.pkl' % num),
       op.join(d,'SNPmat_popNames_%s.txt' % num),
       op.join(d,'UnbinnedImputedSNPSFILE_%s.txt' % num),
       op.join(d,'imputed_MAF_%s.txt' % num),
       op.join(d,'OutFLANK_snps_%s.txt' % num),
       op.join(d,'imputed_dvals_%s.txt' % num),
       op.join(d,'covariances/randmatrices/randsnps')
       )
    filE = op.join(d,'get_dij_%s.py' % num)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [0]:
DIR = "/home/lindb/wbp/outflank_null/"
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d]
files = []
for d in dirs:
    num = op.basename(d).split("_")[1]
    f = op.join(d,'get_dij_%s.py' % num)
    assert op.exists(f)
    files.append(f)
len(files)

In [0]:
DIR = '/home/lindb/wbp/outflank_null/make_null_sets'
if not op.exists(DIR):
    os.makedirs(DIR)
for f in sorted(files):
    num = op.dirname(f).split("/")[-1].split("_")[1]
    text = '''#!/usr/bin/bash
#$ -N inf%s
#$ -S /bin/bash
#$ -V
#$ -j y

cd %s
python %s
''' % (num,
       op.dirname(f),
       f
      )
    filE = op.join(DIR,'make_null_%s.sh' % num)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [0]:
DIR = '/home/lindb/wbp/outflank_null/make_null_sets'
files = [op.join(DIR,f) for f in ls(DIR)]
for f in files:
    #!qsub $f

In [0]:
#check on progress
DIR = '/home/lindb/wbp/outflank_null/'
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d and 'make' not in d]
count = 0
for d in sorted(dirs):
    num = op.basename(d).split("_")[1]
    DIR = op.join(d,'covariances/randmatrices/randsnps')
    if op.exists(DIR):
        #files = [f for f in ls(DIR)]
        #print num, len(files)
        count += 1
print '\n',count

## get dij for random snp sets

In [0]:
DIR = '/home/lindb/wbp/outflank_null/'
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d and 'make' not in d]
for d in dirs:
    num = op.basename(d).split("_")[1]
    DIR = op.join(d, 'covariances/randmatrices/randsnps/')
    files = [op.join(DIR,f) for f in ls(DIR)]
    assert len(files) == 1000

In [0]:
DIR = '/home/lindb/wbp/outflank_null/'
dirs = sorted([op.join(DIR,d) for d in ls(DIR) if 'null' in d and 'make' not in d])
for d in dirs:
    num = op.basename(d).split("_")[1]
    text = '''from __future__ import division
import os
from collections import OrderedDict,Counter
import pandas as pd
import numpy as np
import vcf
from operator import itemgetter
import random
import math
from os import listdir as ls
from os import path as op
import skbio
import shutil
import pickle

#get global minor allele frequencies
print "getting global maf"
glob = pickle.load(open("/home/lindb/wbp/OutFLANK/global_mafs.pkl","rb"))

#get 012 data with population assignments
print "uploading 012"
filE = "%s"                                                          ###############################
data = pickle.load(open(filE,"rb"))

print "merging data frames"
m = pd.read_csv("%s", header=None,index_col=None,sep="\\t")          ###############################
data["pop"] = m[0].tolist()
cols = ["pop"] + [col for col in data.columns if "NODE" in col]
data = data[cols]

#get a list of samps for each population
print "getting samp lists per pop - these samps have been randomly assigned"
pops = np.unique(data["pop"]).tolist()
popDict = OrderedDict()
for samp in data.index:
    pop = data.loc[samp,"pop"]
    if not pop in popDict.keys():
        popDict[pop] = []
    popDict[pop].append(samp)

print "getting pop counts"
popcounts = OrderedDict()
for pop in pops:
    popcounts[pop] = len(popDict[pop])
    
popMAF = pd.read_csv("%s",header=0,index_col=0,sep="\\t")            ###############################

print "calculating dij for 1000 sets of null snps"
DIR = "%s"                                                           ###############################
files = [op.join(DIR,f) for f in ls(DIR)]
assert len(files) == 1000
COUNTS = 0
for f in sorted(files):
    num = op.basename(f).split("_")[2]
    df = pd.read_csv(f, header=None,sep="\\t")
    randomsnps = df[0].tolist()

    dijDict = OrderedDict() 
    for i,locusi in enumerate(randomsnps):
        dijDict[locusi] = OrderedDict()
        qi = glob[locusi] #global maf

        for j,locusj in enumerate(randomsnps):
            if i > j: #i=row, j=col : lower triangle 
                qj = glob[locusj] #global maf

                sums = 0
                for pop in popMAF.columns:
                    qik = popMAF.loc[locusi,pop] #get pop maf
                    qjk = popMAF.loc[locusj,pop] #get pop maf
                    nk = popcounts[pop]

                    sums += (nk/244)*((qik*qjk)-(qi*qj))

                dijDict[locusi][locusj] = sums
            else:
                dijDict[locusi][locusj] = np.nan
    DIR = "%s"                                                       ###############################
    if not op.exists(DIR):
        os.makedirs(DIR)

    filE = op.join(DIR,"outflank_null_%%s_null_DVALS.txt" %% num)   
    with open(filE,'w') as o:
        key0 = dijDict.keys()[0]
        line = '\\t'.join(dijDict[key0].keys()) + str('\\n')
        o.write("%%s" %% line)
        for locusi in dijDict.keys():
            line = str(locusi)+'\\t'+'\\t'.join([str(x) for x in dijDict[locusi].values()]) + str('\\n')
            o.write("%%s" %% line)
    COUNTS += 1
    if COUNTS %% 10 == 0:
        print num, COUNTS


''' % (op.join(d,'SNPmat_HEADERIDX_%s.pkl' % num),
       op.join(d,'SNPmat_popNames_%s.txt' % num),
       op.join(d,'imputed_MAF_%s.txt' % num),
       op.join(d,'covariances/randmatrices/randsnps/'),
       op.join(d,'covariances/randmatrices/randdij/')
      )
    DIR = '/home/lindb/wbp/outflank_null/get_rand_dij'
    if not op.exists(DIR):
        os.makedirs(DIR)
    filE = op.join(DIR,'get_rand_dij_%s.py' % num)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [0]:
DIR = '/home/lindb/wbp/outflank_null/get_rand_dij'
files = [op.join(DIR,f) for f in ls(DIR)]
for f in files:
    num = op.basename(f).split(".")[0].split("_")[-1]
    text = '''#!/usr/bin/bash
#$ -N inf%s
#$ -S /bin/bash
#$ -V
#$ -j y

cd %s
python %s
''' % (num,
       op.dirname(f),
       f
      )
    filE = op.join(DIR,'go_get_rand_dij_%s.sh' % num)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [0]:
DIR = '/home/lindb/wbp/outflank_null/get_rand_dij'
files = [op.join(DIR,f) for f in ls(DIR) if 'sh' in f]
for f in sorted(files):
    !qsub $f

In [0]:
#check on progress
DIR = '/home/lindb/wbp/outflank_null/'
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d and 'make' not in d]
count = 0
COUNT = 0
for d in sorted(dirs):
    num = op.basename(d).split("_")[1]
    DIR = op.join(d,'covariances/randmatrices/randdij/')
    if op.exists(DIR):
        files = [f for f in ls(DIR)]
        #print num, len(files)
        if len(files) == 1000:
            count += 1
            #print op.basename(d)
            #print num
        if len(files) < 1000:
            print op.basename(d), len(files)
        COUNT += 1
print '\n',count
print '\n',COUNT

# is there higher covariance than random sets?

In [0]:
#for each null run of outflank
    #get the distribution of 1000 medians
    #get the median focal dij ('emp')
    #what is the percentile of 'emp' in the distriubtion of 1000 medians?

In [0]:
DIR = '/home/lindb/wbp/outflank_null/'
dirs = sorted([op.join(DIR,d) for d in ls(DIR) if 'null' in d and 'make' not in d])
for d in dirs:
    num = op.basename(d).split("_")[1]
    text = '''from __future__ import division
import os
from collections import OrderedDict,Counter
import pandas as pd
import numpy as np
import vcf
from operator import itemgetter
import random
import math
from os import listdir as ls
from os import path as op
import skbio
import shutil
import pickle

print "getting empirical dij value"
filE = "%s"                                                       ###############################
print filE
DF = pd.read_csv(filE,header=0,index_col=0,sep="\\t")
empdijs = []
for i,locusi in enumerate(DF.index):
    for j,locusj in enumerate(DF.columns):
        if i > j:
            empdijs.append(abs(float(DF.loc[locusi,locusj])))
empmed = np.median(empdijs)

DIR = "%s"                                                        ###############################
files = [op.join(DIR,f) for f in ls(DIR)]
assert len(files) == 1000
randmeds = []
for f in sorted(files):
    df = pd.read_csv(f,header=0,index_col=0,sep="\\t")
    vals = []
    for i,row in enumerate(df.index):
        for j,col in enumerate(df.columns):
            if i > j:
                vals.append(abs(float(df.loc[row,col])))
    med = np.median(vals)
    randmeds.append(med)

for i,val in enumerate(sorted(randmeds)):
    if empmed > val:
        perc = (i+1)/len(randmeds)
        
dataf = pd.DataFrame()
dataf['randmeds'] = sorted(randmeds)
dataf.loc[0,'empmed'] = empmed
dataf.loc[0,'percentile'] = perc
dataf.loc[0,'greater_than_max'] = empmed / max(randmeds)
dataf.loc[0,'greater_than_n5th'] = empmed / sorted(randmeds)[int(round(len(randmeds)*0.95))]
print len(DF.index)
#dataf.loc[0,'len_outliers'] = len(DF.index)

filE = "%s"                                                       ###############################
dataf.to_csv(filE,header=True,index=False,sep="\\t")




''' % (op.join(d,'imputed_dvals_%s.txt' % num),
       op.join(d,'covariances/randmatrices/randdij'),
       op.join(d,'results_%s.txt' % num)
      )
    DIR = '/home/lindb/wbp/outflank_null/get_results'
    if not op.exists(DIR):
        os.makedirs(DIR)
    filE = op.join(DIR,'get_results_%s.py' % num)
    with open(filE,'w') as o:
        o.write("%s" % text)
    

In [0]:
DIR = '/home/lindb/wbp/outflank_null/get_results'
files = [op.join(DIR,f) for f in ls(DIR) if 'sh' not in f]
assert len(files) == 100
for f in sorted(files):
    num = op.basename(f).split(".")[0].split("_")[2]
    text = '''#!/usr/bin/bash
#$ -N inf%s
#$ -S /bin/bash
#$ -V
#$ -j y

cd %s
python %s
''' % (num,
       DIR,
       f
      )
    filE = op.join(DIR,'go_get_results_%s.sh' % num)
    with open(filE,'w') as o:
        o.write("%s" % text)

In [0]:
#check on progress
DIR = '/home/lindb/wbp/outflank_null/'
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d and 'make' not in d]
assert len(dirs) == 100
count = 0
for d in dirs:
    num = op.basename(d).split("_")[1]
    filE = op.join(d,'results_%s.txt' % num)
    if op.exists(filE):
        count += 1
        os.remove(filE)
count

In [0]:
#get the results
DIR = '/home/lindb/wbp/outflank_null/'
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d and 'make' not in d]
assert len(dirs) == 100
percs = []
for d in dirs:
    num = op.basename(d).split("_")[1]
    filE = op.join(d,'results_%s.txt' % num)
    if op.exists(filE):
        df = pd.read_csv(filE,header=0,sep='\t')
        percs.append(df.loc[0,'percentile'])
plt.hist(percs), len(percs)

In [0]:
#get the results
DIR = '/home/lindb/wbp/outflank_null/'
dirs = [op.join(DIR,d) for d in ls(DIR) if 'null' in d and 'make' not in d]
assert len(dirs) == 100
gts = []
for d in dirs:
    num = op.basename(d).split("_")[1]
    filE = op.join(d,'results_%s.txt' % num)
    if op.exists(filE):
        df = pd.read_csv(filE,header=0,sep='\t')
        gts.append(df.loc[0,'greater_than_max'])
    else:
        print filE
plt.hist(gts), len(gts)

In [0]:
np.median(gts),np.mean(gts)

In [0]:
sorted(gts)

In [0]:
np.std(gts)

In [0]:
(10.6-np.mean(gts))/np.std(gts)

In [0]:
10.6/np.median(gts)

In [0]:
10.6/max(gts)

# Using a different method to randomize individuals in populations (just in case)

### get necessary dfs

In [0]:
glob = pickle.load(open("/home/lindb/wbp/OutFLANK/global_mafs.pkl","rb"))

#z12 file created in 06_pca.ipyn
filE = '/home/lindb/wbp/OutFLANK/imputed_z12_maf_swp_trans_z12.pkl'
imp012 = pickle.load(open(filE,'rb'))

#get pop assignment for each samp
print "making stp df"
filE = '/home/lindb/wbp/sampsTOpop.txt'
stp = pd.read_csv(filE,header=0,index_col='sampID',sep="\\t")
stp.sort_index(inplace=True)

#assign samps to pop by random number
samps = stp.index.tolist()
stpnew = pd.DataFrame(stp)
stpnew['runif'] = ""
for row in stp.index:
    stpnew.loc[row,'runif'] = random.random()
stpnew.sort_values(by=['runif'],inplace = True)
stpnew.index = [samp for samp in samps]
stpnew.sort_index(inplace=True)
stpnew.head()

In [0]:
merged = pd.merge(imp012,stpnew,left_index=True,right_index=True) 
cols = ['pop'] + [col for col in merged.columns if 'NODE' in col] 
merged = merged[cols]
merged.sort_index(inplace=True)
merged.head()

In [0]:
merged.shape

In [0]:
cols = [col for col in merged.columns if 'NODE' in col] 
snpmat = merged[cols]
snpmat.head()

In [0]:
pops = pd.DataFrame(merged['pop'].tolist())
pops.head()

In [0]:
DIR = '/home/lindb/wbp/outflank_null/andrews'
if not op.exists(DIR):
    os.makedirs(DIR)

print 'making 1st snpmat' 
filE = op.join(DIR,'SNPmat_HEADERIDX.txt')
snpmat.to_csv(filE,header=True,index=True,sep="\t")

print 'making 2nd snpmat' 
filE2 = op.join(DIR,'SNPmat_noHEADERIDX.txt')
snpmat.to_csv(filE2,header=None,index=False,sep="\t")

print "making pickle"
pfile = op.join(DIR,'SNPmat_HEADERIDX.pkl')
with open(pfile,'wb') as o:
    pickle.dump(snpmat,o,pickle.HIGHEST_PROTOCOL)    

print 'making popfile' 
popfile = op.join(DIR,'SNPmat_popNames.txt')
pops.to_csv(popfile,header=False,index=False,sep="\t")

print 'making locfile'
locfile = op.join(DIR,'SNPmat_locusNames.txt')
cols = pd.DataFrame(snpmat.columns) 
cols.to_csv(locfile,header=False,index=False,sep="\t")

In [0]:
text = '''
library(OutFLANK)

library(data.table)

SNPmat = data.frame(fread('/home/lindb/wbp/outflank_null/andrews/SNPmat_noHEADERIDX.txt',header=F,sep="\\t"))

locusNames = read.csv('/home/lindb/wbp/outflank_null/andrews/SNPmat_locusNames.txt',header=F,sep="\\t")

popNames = read.csv('/home/lindb/wbp/outflank_null/andrews/SNPmat_popNames.txt',header=F,sep="\\t")

FstDataFrame = MakeDiploidFSTMat(SNPmat,locusNames,popNames)

out = OutFLANK(FstDataFrame = FstDataFrame,NumberOfSamples = 8)

df = out$results

outliers = df[which(df$OutlierFlag == 'TRUE'),]

loci = outliers$LocusName

write.table(df,'/home/lindb/wbp/outflank_null/andrews/OutFLANK_results.txt',row.names=F,col.names=T,sep='\\t')

write.table(loci,'/home/lindb/wbp/outflank_null/andrews/OutFLANK_snps.txt',row.names=F,sep='\\t')

print("DONE!")
'''

filE = "/home/lindb/wbp/outflank_null/andrews/do_the_outflank.R" 

with open(filE,'w') as o:
    o.write("%s" % text)

In [0]:
DIR = "/home/lindb/wbp/outflank_null/andrews"
sigs = []
for d in [DIR]:
    files = [op.join(d,'OutFLANK_snps.txt')] 
    if len(files) >0:
        df = pd.read_csv(files[0])
        sigs.append(len(df.index)) 
    else:
        print op.basename(d) 
sigs

# covariances

In [0]:
d = "/home/lindb/wbp/outflank_null/andrews"
#get global minor allele frequencies
print "getting global maf"
glob = pickle.load(open("/home/lindb/wbp/OutFLANK/global_mafs.pkl","rb"))

#get 012 data with population assignments
print "uploading 012"
filE = op.join(d,'SNPmat_HEADERIDX.pkl') 
data = pickle.load(open(filE,"rb"))

print "merging data frames"
m = pd.read_csv(op.join(d,'SNPmat_popNames.txt'), header=None,index_col=None,sep="\t")     
data["pop"] = m[0].tolist()
cols = ["pop"] + [col for col in data.columns if "NODE" in col]
data = data[cols]

#get a list of samps for each population
print "getting samp lists per pop - these samps have been randomly assigned"
pops = np.unique(data["pop"]).tolist()
popDict = OrderedDict()
for samp in data.index:
    pop = data.loc[samp,"pop"]
    if not pop in popDict.keys():
        popDict[pop] = []
    popDict[pop].append(samp)

#get dataframes for each pop based on samples assigned to that pop
print "getting pop-level dfs"
popDictImp = OrderedDict()
for pop in sorted(pops):
    popDictImp[pop]  = data[data.index.isin(popDict[pop])]

#get major and minor allele counts per pop
print "determining mjr mnr counts"
filE = op.join(d,'UnbinnedImputedSNPSFILE.txt')
if not op.exists(filE):
    locCount = 0
    Dict = OrderedDict()
    for locus in data.columns[data.columns != 'pop']:
        Dict[locus] = OrderedDict()
        popCount = 0
        for pop in sorted(pops):

            zero = popDictImp[pop][locus].tolist().count(0) #count the first homozygotes
            one = popDictImp[pop][locus].tolist().count(1) #count the heterozygotes
            two = popDictImp[pop][locus].tolist().count(2) #count the second homozygotes        

            #012 counts global minor allele
            A1 = 2*zero + one
            A2 = 2*two + one

            #no need to distinguish A1 > A2, since MAF within pop refers to global minor allele

            if len(Dict[locus].keys()) == 0:
                Dict[locus]["A1"] = OrderedDict()
                Dict[locus]["A2"] = OrderedDict()
            Dict[locus]["A1"][pop] = A1
            Dict[locus]["A2"][pop] = A2
            #break
        locCount += 1
        if locCount % 1000 == 0:
            print locCount

    print "writing mjr mnr count file"
    with open(filE, "w") as o:
        line = "\t".join([str(pop) for pop in Dict[Dict.keys()[0]][Dict[Dict.keys()[0]].keys()[0]].keys()]) + str("\n")
        o.write("%s" % line)
        for locus in Dict.keys():
            for allele in Dict[locus].keys():
                line = str(locus) + "\t" + "\t".join([str(x) for x in Dict[locus][allele].values()]) + str("\n")
                #print locus,allele,line
                o.write("%s" % line)
else:
    print "mjr mnr count file already exists, skipping calculation"

#get allele counts by pop - first locus = counts of 0 allele, second = counts of 2 allele
    #012 counts global minor allele
counts = pd.read_csv(filE,header=0,index_col=0,sep="\t")

loci = np.unique(counts.index).tolist()
print "getting pop-level maf"
filE = op.join(d,'imputed_MAF.txt')
if not op.exists(filE):
    loccount = 0
    mafDict = OrderedDict()
    for locus in loci:
        mafDict[locus] = OrderedDict()
        DATA = pd.DataFrame(counts.loc[locus,:])
        DATA.index = ["major","minor"]
        for pop in DATA.columns:
            MAF = DATA.loc["minor",pop]/sum(DATA[pop]) #get allele freq corresponding to global minor allele
            mafDict[locus][pop] = MAF
        loccount += 1
        if loccount % 1000 == 0:
            print loccount

    print "writing pop-level maf"
    with open(filE,"w") as o:
        text = "\t".join([x for x in mafDict[mafDict.keys()[0]].keys()]) + "\n"
        o.write("%s" % text)
        print text
        count = 0
        for locus in mafDict.keys():
            text = locus + "\t" + "\t".join([str(x) for x in mafDict[locus].values()]) + "\n"
            o.write("%s" % text)
            count += 1
else:
    print "pop-level maf already created, skipping calculation"

print "reading pop-level maf"
impMAF = pd.read_csv(filE,header=0,index_col=0,sep="\t")
##############################################################################################

#get outlier snps
print "getting outlier snps"
df = pd.read_csv(op.join(d,'OutFLANK_snps.txt'),header=0,sep="\t")  
outliersnps = df["x"].tolist()

#do pairwise to get Dij for outliers
print "getting pop counts"
popcounts = OrderedDict()
for pop in pops:
    popcounts[pop] = len(popDict[pop])

print "getting dij for outliers"
dijDict = OrderedDict() 
icount = 0
for i,locusi in enumerate(outliersnps):
    dijDict[locusi] = OrderedDict()
    qi = glob[locusi] #global maf
    
    for j,locusj in enumerate(outliersnps):
        if i > j: #i=row, j=col : lower triangle 
            qj = glob[locusj] #global maf
            
            sums = 0
            for pop in impMAF.columns:
                qik = impMAF.loc[locusi,pop] #get pop maf
                qjk = impMAF.loc[locusj,pop] #get pop maf
                nk = popcounts[pop]
                
                sums += (nk/244)*((qik*qjk)-(qi*qj))

            dijDict[locusi][locusj] = sums
        else:
            dijDict[locusi][locusj] = np.nan
    icount += 1
    if icount % 10 == 0:
        print icount
#write out the file
print "writing dij for outliers"
rowcount = 0
filE = op.join(d,'imputed_dvals.txt')
with open(filE,"w") as o:
    key0 = dijDict.keys()[0]
    line = "\t".join(dijDict[key0].keys()) + str("\n")
    o.write("%s" % line)
    for locusi in dijDict.keys():
        line = str(locusi)+"\t"+"\t".join([str(x) for x in dijDict[locusi].values()]) + str("\n")
        o.write("%s" % line)
dvals = pd.read_csv(filE,header=0,index_col=0,sep="\t")

#get global expected heterozygosity and bins
print "getting global het bins"
filE = "/home/lindb/wbp/OutFLANK/Hexp_by_snp_withbins.txt"
H = pd.read_csv(filE, header=0,index_col=0,sep="\t")
H.index = [snp for snp in H["locus"].tolist()]

#get a dataframe with the outlier loci and their bins
print "getting bins for outlier data"
outlierdata = pd.DataFrame(H[H["locus"].isin(outliersnps)])
outlierdata.index = [snp for snp in outlierdata["locus"].tolist()]

print "determining non-outliers for null draws"
nonsigs = set(H.index.tolist()) - set(outlierdata.index.tolist())
nonsigs = [x for x in nonsigs]
nonsigdata = pd.DataFrame(H[H["locus"].isin(nonsigs)])

#how many random snps from each bin?
print "creating binCounter for outlier loci"
binCounter = Counter()
for row in outlierdata.index:
    binCounter[outlierdata.loc[row,"bin"]] += 1

folder = op.join(d,'covariances/randmatrices/randsnps')
if not op.exists(folder):
    os.makedirs(folder)

#make 1000 dataframes of random snps of equal size
print "making 1000 dfs of random snps"
for i in range(1000):
    snps = []        
    for binn in binCounter.keys():
        data = nonsigdata[nonsigdata["bin"] == binn]

        [snps.append(snp) for snp in random.sample(data.index,binCounter[binn])]

    filE = op.join(folder,"outflank_null_%s_randsnps.txt" % (str(i).zfill(4)))
    df = pd.DataFrame(snps)
    df.to_csv(filE,header=False,index=False,sep="\t")

In [0]:
DIR = '/home/lindb/wbp/outflank_null/andrews/'
for d in [DIR]:
    DIR = op.join(d, 'covariances/randmatrices/randsnps/')
    files = [op.join(DIR,f) for f in ls(DIR)]
    assert len(files) == 1000

In [0]:
d = '/home/lindb/wbp/outflank_null/andrews/'

#get global minor allele frequencies
print "getting global maf"
glob = pickle.load(open("/home/lindb/wbp/OutFLANK/global_mafs.pkl","rb"))

#get 012 data with population assignments
print "uploading 012"
filE = op.join(d,'SNPmat_HEADERIDX.pkl')
data = pickle.load(open(filE,"rb"))

print "merging data frames"
m = pd.read_csv(op.join(d,'SNPmat_popNames.txt'), header=None,index_col=None,sep="\t")    
data["pop"] = m[0].tolist()
cols = ["pop"] + [col for col in data.columns if "NODE" in col]
data = data[cols]

#get a list of samps for each population
print "getting samp lists per pop - these samps have been randomly assigned"
pops = np.unique(data["pop"]).tolist()
popDict = OrderedDict()
for samp in data.index:
    pop = data.loc[samp,"pop"]
    if not pop in popDict.keys():
        popDict[pop] = []
    popDict[pop].append(samp)

print "getting pop counts"
popcounts = OrderedDict()
for pop in pops:
    popcounts[pop] = len(popDict[pop])
    
popMAF = pd.read_csv(op.join(d,'imputed_MAF.txt'),header=0,index_col=0,sep="\t")    

print "calculating dij for 1000 sets of null snps"
DIR = op.join(d,'covariances/randmatrices/randsnps/')
files = [op.join(DIR,f) for f in ls(DIR)]
assert len(files) == 1000
COUNTS = 0
for f in sorted(files):
    num = op.basename(f).split("_")[2]
    df = pd.read_csv(f, header=None,sep="\t")
    randomsnps = df[0].tolist()

    dijDict = OrderedDict() 
    for i,locusi in enumerate(randomsnps):
        dijDict[locusi] = OrderedDict()
        qi = glob[locusi] #global maf

        for j,locusj in enumerate(randomsnps):
            if i > j: #i=row, j=col : lower triangle 
                qj = glob[locusj] #global maf

                sums = 0
                for pop in popMAF.columns:
                    qik = popMAF.loc[locusi,pop] #get pop maf
                    qjk = popMAF.loc[locusj,pop] #get pop maf
                    nk = popcounts[pop]

                    sums += (nk/244)*((qik*qjk)-(qi*qj))

                dijDict[locusi][locusj] = sums
            else:
                dijDict[locusi][locusj] = np.nan
    DIR = op.join(d,'covariances/randmatrices/randdij/')
    if not op.exists(DIR):
        os.makedirs(DIR)

    filE = op.join(DIR,"outflank_null_andrew_%s_DVALS.txt" % num)   
    with open(filE,'w') as o:
        key0 = dijDict.keys()[0]
        line = '\t'.join(dijDict[key0].keys()) + str('\n')
        o.write("%s" % line)
        for locusi in dijDict.keys():
            line = str(locusi)+'\t'+'\t'.join([str(x) for x in dijDict[locusi].values()]) + str('\n')
            o.write("%s" % line)
    COUNTS += 1
    if COUNTS % 10 == 0:
        print COUNTS

In [0]:
d = '/home/lindb/wbp/outflank_null/andrews'

print "getting empirical dij value"
filE = op.join(d,'imputed_dvals.txt')
print filE
DF = pd.read_csv(filE,header=0,index_col=0,sep="\t")
empdijs = []
for i,locusi in enumerate(DF.index):
    for j,locusj in enumerate(DF.columns):
        if i > j:
            empdijs.append(abs(float(DF.loc[locusi,locusj])))
empmed = np.median(empdijs)

DIR = op.join(d,'covariances/randmatrices/randdij')
files = [op.join(DIR,f) for f in ls(DIR)]
assert len(files) == 1000
randmeds = []
for f in sorted(files):
    df = pd.read_csv(f,header=0,index_col=0,sep="\t")
    vals = []
    for i,row in enumerate(df.index):
        for j,col in enumerate(df.columns):
            if i > j:
                vals.append(abs(float(df.loc[row,col])))
    med = np.median(vals)
    randmeds.append(med)

for i,val in enumerate(sorted(randmeds)):
    if empmed > val:
        perc = (i+1)/len(randmeds)
        
dataf = pd.DataFrame()
dataf['randmeds'] = sorted(randmeds)
dataf.loc[0,'empmed'] = empmed
dataf.loc[0,'percentile'] = perc
dataf.loc[0,'greater_than_max'] = empmed / max(randmeds)
dataf.loc[0,'greater_than_n5th'] = empmed / sorted(randmeds)[int(round(len(randmeds)*0.95))]
print len(DF.index)
#dataf.loc[0,'len_outliers'] = len(DF.index)

filE = op.join(d,'results.txt')
dataf.to_csv(filE,header=True,index=False,sep="\t")

In [0]:
dataf